## This file is to extract contact information with some constrant
* **Constraints:**  
 * length, gives the frame length  
 * ref_chain gives the reference chain, it takes values of either 'Ab' or 'Ag'
For example, *ref_chain = 'Ag', length = 2*, means we try to find consecutive amino acids with length 2 in the antigen, with the largest contact number among all consecutive amino acids with the same length in the antigen, lets call those consecutive amino acids, aa_max.
**Here, we give all aa_max with length no larger than *length***

### Group
* **Inputs:**  
 * four_coordinates, a list contains all the interations of one pdb  
* **Returns:**  
 * grouped, a dictionary of four coordiantes, with keys h1HA, h2HA,...

In [2]:
def group(four_coordinates):
    grouped = {}
    for i in four_coordinates:
        if i[0] not in grouped:
            grouped[i[0]] = [i]
        else:
            grouped[i[0]].append(i)
    return grouped

### extract_aa_pos
* **Inputs:**  
 * grouped_list, a list of four coordinates, for example, a list of four coordinates of 'h1HA' of '1adq'.  
 * ref_chain, a string, takes values as either 'Ab' or 'Ag'.  
* **Returns:**  
 * aa_pos, a list contains the sorted list of amino acid positions without repetition either of antigen chain or antibody chain.


In [3]:
def extract_aa_pos(grouped_list, ref_chain = 'Ag'):    
    if ref_chain == 'Ab':
        ind = 1
    if ref_chain == 'Ag':
        ind = 2
    aa_pos = []
    for i in grouped_list:
        if i[ind] not in aa_pos:
            aa_pos.append(i[ind])
    aa_pos.sort()    
    return aa_pos

### extract_aa_consecutive_pos
* **Inputs:*  
 * aa_pos, a list of amino acid positions without repeat  
 * length, an integer, give the length of the reference frame  
* **Retruns:**  
 * aa_consecutive_pos, a list aa_positions which are consecutive with length length, in the form [[1, 2, 3], [3, 4, 5],...]

In [4]:
def extract_aa_consecutive_pos(aa_pos, length):
    aa_consecutive_pos = []
    if len(aa_pos) >= length:
        for i in range(len(aa_pos) - length + 1):
            if aa_pos[i + length - 1] -  aa_pos[i] == length - 1:
                aa_consecutive_pos.append(aa_pos[i : i + length ])              
        
    return aa_consecutive_pos

### extract_aa_max
* **Inputs:**
 * aa_consecutive_pos, a list of list, gives the position of the amino acids in the form of [[1, 2, 3], [3, 4, 5],...].  
 * four_coordinates, a list of four coordinates, to which the aa_consecutive_pos is related.  
 * ref_chain, a string, takes values as either 'Ab' or 'Ag'  
* **Returns:**  
 * aa_max, a list of aa positions, corresponding to the biggest contact number among all the aa_consecutive_pos.  
 * contact, an integer, gives the total number of conatact corresponding to aa_max

In [5]:
def extract_aa_max(aa_consecutive_pos, four_coordinates, ref_chain = 'Ag'):
    contact = 0
    aa_max = None    
    if ref_chain == 'Ab':
        ind = 1
    if ref_chain == 'Ag':
        ind = 2
    for i in aa_consecutive_pos:
        s = 0
#        aa_max_temp = []
        for j in four_coordinates:
            if j[ind] in i:
#                aa_max_temp.append(j)
                s += j[3]
        if contact <= s:
            contact = s
            aa_max = i
    return aa_max, contact

### Main
* **Inputs:**  
 * *contact*, a list of four coordinates for one pdb.  
 * *length*, an integer, gives the maximum length of the consecutive amino acids  
 * *ref_chain*, a string, takes values as either 'Ab' or 'Ag'.  
* **Returns:**  . 
 * *grouped_aa_max_contact*, a dictionary, in the form of {'h1HA': [[[16], [16, 17], None], [21, 37, 0]], ...}. It means for this pdb file, with *length* no larger than 3, and *ref_chain = 'Ag'*, amino acid *16* is the one with the largest contact number, 21; amino acids *[16, 17]* are the consecutive 2 amino acides with the largest contact number 37; there is no consecutive amino acids with length 3 in the antigen chain, which conatact with CDRh1 as well.
 

In [6]:
def main(contact, length = 3, ref_chain = 'Ag'):
    grouped = group(contact)
    grouped_aa_max_contact = {}
    for i in grouped:
        if grouped[i] != [[]]:
            aa_pos = extract_aa_pos(grouped[i], ref_chain)
            total_aa_consecutive_pos = []
            total_aa_max = []
            total_contact_max = []
            for j in range(length):
               total_aa_consecutive_pos.append(extract_aa_consecutive_pos(aa_pos, j+1))
            for k in total_aa_consecutive_pos:
               aa_max, contact = extract_aa_max(k, grouped[i], ref_chain)
               total_aa_max.append(aa_max)
               total_contact_max.append(contact)
            grouped_aa_max_contact[i] = [total_aa_max, total_contact_max]
    return grouped_aa_max_contact

### add_percentage 
**To calculate the percentage of contact each frame length takes of the total contacts in the CDR**
* **Inputs:**  
 * *DataExtract_pdb*, a dictionary, extracted data for pdb in the form of {'h1HA': [[30], None, None, None], [17, 0, 0, 0], [0.532, ...]],...}  
 * *Four_coordinates*, a list of four coordinates, from the same pdf as DataExtract_pdb, in the form of [['h1HA', 30, 16, 7],...]
* **Returns:**  
 * *DataExtract_pdb*, a dictionary, in the form of {'h1HA': [[[30], None, None, None], [17, 0, 0, 0], [40%, ...]],...}


In [7]:
def add_percentage(DataExtract_pdb, Four_coordinates_pdb):
    for i in DataExtract_pdb:
        s = 0 
        for j in Four_coordinates_pdb:
            if i == j[0]:
                s += j[3]
        if s != 0:
            DataExtract_pdb[i].append([round(k/s, 3) for k in DataExtract_pdb[i][1]])
    return DataExtract_pdb

### Do some preparation of the working directory

In [8]:
import os
os.getcwd()
os.chdir("C:\\Users\\leo\\Documents\\Research\\Database\\Humnized antibody\\PDB DATA\\Homo Sapiens with paptide 5+\\structure")
os.listdir()

['1adq.pdb',
 '1bvk.pdb',
 '1dee.pdb',
 '1g9m.pdb',
 '1g9n.pdb',
 '1gc1.pdb',
 '1h0d.pdb',
 '1hez.pdb',
 '1i9r.pdb',
 '1ikf.pdb',
 '1iqd.pdb',
 '1jps.pdb',
 '1mcc.pdb',
 '1mcd.pdb',
 '1mce.pdb',
 '1mcf.pdb',
 '1mch.pdb',
 '1n0x.pdb',
 '1n8z.pdb',
 '1nl0.pdb',
 '1q1j.pdb',
 '1rzj.pdb',
 '1rzk.pdb',
 '1tjg.pdb',
 '1tjh.pdb',
 '1tji.pdb',
 '1tzg.pdb',
 '1u8h.pdb',
 '1u8i.pdb',
 '1u8j.pdb',
 '1u8k.pdb',
 '1u8l.pdb',
 '1u8m.pdb',
 '1u8n.pdb',
 '1u8o.pdb',
 '1u8p.pdb',
 '1u8q.pdb',
 '1u91.pdb',
 '1u92.pdb',
 '1u93.pdb',
 '1u95.pdb',
 '1uj3.pdb',
 '1w72.pdb',
 '1yyl.pdb',
 '1yym.pdb',
 '2b0s.pdb',
 '2b1a.pdb',
 '2b1h.pdb',
 '2b4c.pdb',
 '2cmr.pdb',
 '2dd8.pdb',
 '2eiz.pdb',
 '2eks.pdb',
 '2f5b.pdb',
 '2fec.pdb',
 '2fed.pdb',
 '2fee.pdb',
 '2fjg.pdb',
 '2fjh.pdb',
 '2fx7.pdb',
 '2fx8.pdb',
 '2fx9.pdb',
 '2ghw.pdb',
 '2h3n.pdb',
 '2h9g.pdb',
 '2hfg.pdb',
 '2i5y.pdb',
 '2i60.pdb',
 '2j6e.pdb',
 '2nxy.pdb',
 '2nxz.pdb',
 '2ny0.pdb',
 '2ny1.pdb',
 '2ny2.pdb',
 '2ny3.pdb',
 '2ny4.pdb',
 '2ny5.pdb',

### load the out put of AAC-1

In [9]:
import json
with open('contact_homo', 'r') as f:
    contact_homo = json.load(f)

### Extract data from the file loaded above

In [10]:
DataExtract_homo = {}
for i in contact_homo:
    DataExtract_homo[i] = main(contact_homo[i], length = 4, ref_chain = 'Ag')
    add_percentage(DataExtract_homo[i], contact_homo[i])

In [11]:
DataExtract_homo

{'1adq': {'h1HA': [[[17], [16, 17], None, None],
   [7, 14, 0, 0],
   [0.412, 0.824, 0.0, 0.0]],
  'h2HA': [[[16], [15, 16], None, None],
   [9, 11, 0, 0],
   [0.818, 1.0, 0.0, 0.0]],
  'h3HA': [[[196], [196, 197], [196, 197, 198], None],
   [17, 20, 30, 0],
   [0.415, 0.488, 0.732, 0.0]],
  'l2LA': [[[195], None, None, None], [6, 0, 0, 0], [0.429, 0.0, 0.0, 0.0]]},
 '1bvk': {'h1BC': [[[115], [115, 116], None, None],
   [4, 7, 0, 0],
   [0.571, 1.0, 0.0, 0.0]],
  'h1EF': [[[115], [115, 116], None, None],
   [8, 10, 0, 0],
   [0.8, 1.0, 0.0, 0.0]],
  'h2BC': [[[118], [117, 118], [116, 117, 118], None],
   [13, 18, 24, 0],
   [0.542, 0.75, 1.0, 0.0]],
  'h2EF': [[[118], [117, 118], [116, 117, 118], None],
   [7, 11, 15, 0],
   [0.467, 0.733, 1.0, 0.0]],
  'h3BC': [[[120], [22, 23], [21, 22, 23], None],
   [10, 13, 20, 0],
   [0.227, 0.295, 0.455, 0.0]],
  'h3EF': [[[120], [119, 120], [118, 119, 120], [117, 118, 119, 120]],
   [7, 11, 17, 18],
   [0.184, 0.289, 0.447, 0.474]],
  'l1AC': [

### Save the results

In [12]:
# with open('DataExtract_homo', 'w') as f:
#     json.dump(DataExtract_homo,f)

### extract_four_coordinates_aa_pos
* **Inputs:**  
 * *DataExtract_pdb*,  a dictionary, extracted data for one pdb in the form of {'h1HA': [[30], None, None, None], [17, 0 0, 0], [0.532, ...]],...}  
 * *Four_coordinates_pdb*, a list of four coordinates, from the same pdf as DataExtract_pdb, in the
        form of [['h1HA', 30, 16, 7],...].
 * *frame_length*, an integer, gives the length of the reference frame  
 * *ref_chain*, a string takes values of 'Ag' or 'Ab', gives the type of the reference chain
* **Returns:**  
 * *extract_four_coordinates*, a dictionary in the form of a dictionary, in the form of {'h1HA':[['h1HA', 30, 16, 7],...], ...}, *for extract_four_coordinates['h1HA']*, it contains all the contacts constrained by the frame length in the form of four coordinates in h1HA.
 * *aa_pos_correspondence_framed*,  a dictionary, in the form of {'h1HA': [[30], [16, 17], 14, 0.824],...}, where the paratope aa or epitope aa is arranged according to the arranged frame,14 stands for the contact number, 0.824 stands for the ratio 14 is to the total contact number of CDRh1. it corresponds to the same pdb as DataExtract_pdb
 

In [13]:
def extract_four_coordinates_aa_pos(DataExtract_pdb, Four_coordinates_pdb, frame_length, ref_chain = 'Ag'):
    if ref_chain == 'Ab':
        ind = 1
    if ref_chain == 'Ag':
        ind = 2
        
    extract_four_coordinates = {}
    for i in DataExtract_pdb:
        extract_four_coordinates[i] = []
        for j in Four_coordinates_pdb:
            if i == j[0] and DataExtract_pdb[i][0][frame_length-1] != None and (j[ind] in DataExtract_pdb[i][0][frame_length-1]):
                 extract_four_coordinates[i].append(j)
        if extract_four_coordinates[i] == []:
            del extract_four_coordinates[i]
            
    aa_pos_correspondence_framed = {}       
    for i in extract_four_coordinates:
        aa_pos_correspondence_framed[i] = [None, None, None, None]
        temp = []
        extract_four_coordinates[i].sort(key = lambda x: x[ind])
        for j in extract_four_coordinates[i]:
            if j[3-ind] not in temp:
                temp.append(j[3-ind])
        aa_pos_correspondence_framed[i][ind-1] = DataExtract_pdb[i][0][frame_length-1]
        aa_pos_correspondence_framed[i][2-ind] = temp
        aa_pos_correspondence_framed[i][2] = DataExtract_pdb[i][1][frame_length-1]
        aa_pos_correspondence_framed[i][3] = DataExtract_pdb[i][2][frame_length-1]
               
    return extract_four_coordinates, aa_pos_correspondence_framed

### map_to_aa
* **Inputs:**  
 * aa_pos_correspondence_framed, the return of *extract_four_coordinates_aa_pos*.
 * seq_pdb, a dictionary, gives the sequence of a dictionary, in the form of  {'H': ['GLU', 'VAL',...],...}, it should be of the same pdb as aa_pos_correspondence_framed.
* **Returns:**  
 * aa_framed,  aa_framed, a dictionary, gives the contacting paratope and epitope amino acids corresponding to the infromation given by *aa_pos_correspondence_framed* in the form of {'h1HA':[[ASP],[SER, ARG],...], ...} if {'h1HA':[[ASP],[SER, O, ARG],...], ...}, O stands for insertion(s).


In [14]:
def map_to_aa(aa_pos_correspondence_framed, seq_pdb):
    aa_framed = {}
    for i in aa_pos_correspondence_framed:
        #initiate the returned values
        aa_framed[i] = [None, None, None, None]
        
        antibody_chain = i[2]        
        antibody_temp = []        
        tracker = aa_pos_correspondence_framed[i][0][0]
        for j in aa_pos_correspondence_framed[i][0]:
            if j - tracker <= 1:
                antibody_temp.append(seq_pdb[antibody_chain][j])
                tracker = j 
            else:
                antibody_temp.extend(['O', seq_pdb[antibody_chain][j]])
                tracker = j 
            
        antigen_chain = i[3]
        antigen_temp = []
        tracker = aa_pos_correspondence_framed[i][1][0]
        for k in aa_pos_correspondence_framed[i][1]:
            if k - tracker <= 1:
                antigen_temp.append(seq_pdb[antigen_chain][k])
                tracker = k 
            else:
                antigen_temp.extend(['O', seq_pdb[antigen_chain][k]])
                tracker = k
        
        aa_framed[i][0] = antibody_temp
        aa_framed[i][1] = antigen_temp
        aa_framed[i][2] = aa_pos_correspondence_framed[i][2]
        aa_framed[i][3] = aa_pos_correspondence_framed[i][3]
    
    return aa_framed

### Import some required data

In [15]:
with open('seq_homo', 'r') as f:
    seq_homo = json.load(f)

### Run

In [17]:
four_coordinates_framed = {}
aa_pos_correspondence_framed = {}
aa_correspondence_framed = {}

for i in DataExtract_homo:
    four_coordinates_framed[i], aa_pos_correspondence_framed[i] = extract_four_coordinates_aa_pos(DataExtract_homo[i],
                        contact_homo[i], frame_length = 2, ref_chain = 'Ag')
    aa_correspondence_framed[i] = map_to_aa(aa_pos_correspondence_framed[i], seq_homo[i])

### Take a look at the results

In [19]:
aa_correspondence_framed['1adq']

{'h1HA': [['ASP'], ['SER', 'ARG'], 14, 0.824],
 'h2HA': [['TRP'], ['ILE', 'SER'], 11, 1.0],
 'h3HA': [['ARG', 'SER', 'TYR', 'VAL'], ['ASN', 'HIS'], 20, 0.488]}

### Save the results

In [20]:
# with open('ParaEpi_framed', 'w') as f：
#     json.dump(aa_correspondence_framed, f)
# with open('ParaEpi_pos_framed', 'w') as f：
#     json.dump(aa_pos_correspondence_framed, f)
# with open('ParaEpi_fcdn_framed', 'w') as f:
#     json.dump(four_coordinates_framed, f)
